In [16]:
class RNN:
	def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
		# hyper parameters
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.seq_length = seq_length
		self.learning_rate = learning_rate

		# weights for input -> header layer
		self.w_layer1 = np.random.uniform(-np.sqrt(1. / vocab_size), np.sqrt(1. / vocab_size), (hidden_size, vocab_size))
		# weights for header -> header layer
		self.w_layer3 = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size), (vocab_size, hidden_size))
		# weights for header -> output layer
		self.w_layer2 = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size), (hidden_size, hidden_size))
		self.b_layer2 = np.zeros((hidden_size, 1))  # bias for hidden layer
		self.b_layer3 = np.zeros((vocab_size, 1))  # bias for output

		# memory vars for Adagrad - https://machinelearningmastery.com/gradient-descent-with-adagrad-from-scratch/
		# ignore if you implement another approach
		self.mem_w_layer1 = np.zeros_like(self.w_layer1)
		self.mem_w_layer2 = np.zeros_like(self.w_layer2)
		self.mem_w_layer3 = np.zeros_like(self.w_layer3)
		self.mem_b_layer2 = np.zeros_like(self.b_layer2)
		self.mem_b_layer3 = np.zeros_like(self.b_layer3)

	def softmax(self, x):
		p = np.exp(x - np.max(x))
		return p / np.sum(p)

	def forward(self, inputs, hprev):
		hot_encoded_chars, activations, net, prediction = {}, {}, {}, {}
		activations[-1] = np.copy(hprev)
		for t in range(len(inputs)):
			hot_encoded_chars[t] = np.zeros((self.vocab_size, 1))
			hot_encoded_chars[t][inputs[t]] = 1
			activations[t] = np.tanh(np.dot(self.w_layer1, hot_encoded_chars[t]) + np.dot(self.w_layer2, activations[t - 1]) + self.b_layer2)
			# taking the dot product between the activation and the weight matrix -- this is called the "net input" to the current layer
			net[t] = np.dot(self.w_layer3, activations[t]) + self.b_layer3  # unnormalised log probs for next char
			prediction[t] = self.softmax(net[t])
		return hot_encoded_chars, activations, prediction

	def backward(self, hot_encoded_chars, activations, predictions, targets):
		# backward pass: compute gradients going backwards
		d_w_layer1, d_w_layer2, d_w_layer3 = np.zeros_like(self.w_layer1), np.zeros_like(self.w_layer2), np.zeros_like(self.w_layer3)
		d_b_layer2, d_b_layer3 = np.zeros_like(self.b_layer2), np.zeros_like(self.b_layer3)
		dhnext = np.zeros_like(activations[0])

		for t in reversed(range(self.seq_length)):
			d_prediction = np.copy(predictions[t])
			d_prediction[targets[t]] -= 1  # backprop into y

			d_b_layer3 += d_b_layer3
			# dh includes gradient from two sides, next cell and current output
			dh = np.dot(self.w_layer3.T, d_prediction) + dhnext  # backprop into h

			derivative_dir = (1 - activations[t] * activations[t]) * dh  # derivative_dir is the term used in many equations
			d_b_layer2 += derivative_dir

			d_w_layer1 += np.dot(derivative_dir, hot_encoded_chars[t].T)
			d_w_layer2 += np.dot(derivative_dir, activations[t - 1].T)
			d_w_layer3 += np.dot(d_prediction, activations[t].T)

			# pass the gradient from next cell to the next iteration.
			dhnext = np.dot(self.w_layer2.T, derivative_dir)
		# clip to mitigate exploding gradients
		for dparam in [d_w_layer1, d_w_layer2, d_w_layer3, d_b_layer2, d_b_layer3]:
			np.clip(dparam, -5, 5, out=dparam)
		return d_w_layer1, d_w_layer2, d_w_layer3, d_b_layer2, d_b_layer3

	def loss(self, ps, targets):
		"""loss for a sequence"""
		# calculate cross-entrpy loss
		return sum(-np.log(ps[t][targets[t], 0]) for t in range(self.seq_length))

	def update_model(self, dU, dW, dV, db, dc):
		# parameter update with adagrad
		for param, dparam, mem in zip([self.w_layer1, self.w_layer2, self.w_layer3, self.b_layer2, self.b_layer3],
									  [dU, dW, dV, db, dc],
									  [self.mem_w_layer1, self.mem_w_layer2, self.mem_w_layer3, self.mem_b_layer2, self.mem_b_layer3]):
			mem += dparam * dparam
			param += -self.learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

	def sample(self, h, seed_ix, n):
		"""
		sample a sequence of integers from the model
		h is memory state, seed_ix is seed letter from the first time step
		"""
		x = np.zeros((self.vocab_size, 1))
		x[seed_ix] = 1
		ixes = []
		for t in range(n):
			h = np.tanh(np.dot(self.w_layer1, x) + np.dot(self.w_layer2, h) + self.b_layer2)
			y = np.dot(self.w_layer3, h) + self.b_layer3
			p = np.exp(y) / np.sum(np.exp(y))
			ix = np.random.choice(range(self.vocab_size), p=p.ravel())
			x = np.zeros((self.vocab_size, 1))
			x[ix] = 1
			ixes.append(ix)
		return ixes

	def train(self, data_reader):
		iter_num = 0
		threshold = 0.01
		smooth_loss = -np.log(1.0 / data_reader.vocab_size) * self.seq_length
		while (smooth_loss > threshold):
			if data_reader.just_started():
				hprev = np.zeros((self.hidden_size, 1))
			inputs, targets = data_reader.next_batch()
			hot_encoded_chars, activations, predictions = self.forward(inputs, hprev)
			dU, dW, dV, db, dc = self.backward(hot_encoded_chars, activations, predictions, targets)
			loss = self.loss(predictions, targets)
			self.update_model(dU, dW, dV, db, dc)
			smooth_loss = smooth_loss * 0.999 + loss * 0.001
			hprev = activations[self.seq_length - 1]
			if not iter_num % 500:
				sample_ix = self.sample(hprev, inputs[0], 200)
				print(''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
				print("\n\niter :%d, loss:%f" % (iter_num, smooth_loss))
			iter_num += 1

	def predict(self, data_reader, start, n):
		# initialize input vector
		x = np.zeros((self.vocab_size, 1))
		chars = [ch for ch in start]
		ixes = []
		for i in range(len(chars)):
			ix = data_reader.char_to_ix[chars[i]]
			x[ix] = 1
			ixes.append(ix)

		h = np.zeros((self.hidden_size, 1))
		# predict next n chars
		for t in range(n):
			h = np.tanh(np.dot(self.w_layer1, x) + np.dot(self.w_layer2, h) + self.b_layer2)
			y = np.dot(self.w_layer3, h) + self.b_layer3
			p = np.exp(y) / np.sum(np.exp(y))
			ix = np.random.choice(range(self.vocab_size), p=p.ravel())
			x = np.zeros((self.vocab_size, 1))
			x[ix] = 1
			ixes.append(ix)
		txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
		return txt

In [23]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer
        self.c = np.zeros((vocab_size, 1)) # bias for output
        
        # memory vars for adagrad, 
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
        
    def forward(self, inputs, hprev):
            xs, hs, os, ycap = {}, {}, {}, {}
            hs[-1] = np.copy(hprev)
            for t in range(len(inputs)):
                xs[t] = np.zeros((self.vocab_size,1))
                xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
                hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b) # hidden state
                os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
                ycap[t] = self.softmax(os[t]) # probs for next char
            return xs, hs, ycap
        
    def backward(self, xs, hs, ps, targets):
            # backward pass: compute gradients going backwards
            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):
                dy = np.copy(ps[t])
                #through softmax
                dy[targets[t]] -= 1 # backprop into y
                #calculate dV, dc
                dV += np.dot(dy, hs[t].T)
                dc += dc
                #dh includes gradient from two sides, next cell and current output
                dh = np.dot(self.V.T, dy) + dhnext # backprop into h
                # backprop through tanh non-linearity 
                dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
                db += dhrec
                #calculate dU and dW
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                #pass the gradient from next cell to the next iteration.
                dhnext = np.dot(self.W.T, dhrec)
            # clip to mitigate exploding gradients
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam) 
            return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
            """loss for a sequence"""
            # calculate cross-entrpy loss
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = np.zeros((self.vocab_size,1))
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = np.zeros((self.vocab_size,1))
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0
            threshold = 0.05
            smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.seq_length
            while (smooth_loss > threshold):
                if data_reader.just_started():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                smooth_loss = smooth_loss*0.999 + loss*0.001
                hprev = hs[self.seq_length-1]
                if not iter_num%500:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
                iter_num += 1

    def predict(self, data_reader, start, n):
        #initialize input vector
        x = np.zeros((self.vocab_size,1))
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = np.zeros((self.vocab_size,1))
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [24]:
seq_length = 25
data_reader = DataReader("texts/input.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

TstzAwBpzs.BhwT!
odvyW sR.-WjRisWPWx/Y sB)P)cjS)lrppthPtcmNsus!zRC !PsoiIdcs?'ppkvtNIrmlPa-aqulYll?rz,qLPsu.pcntrsTWSsbL)!,gC?-NRqP'IL s/ci-jo-ymcvs sglebcedgYrwU,fyT?SI',Il(PmigjsrTwg sUia)pi.d/sPcB'


iter :0, loss:98.295346
h ne hnpss ahahey insald int, Cl iod ure uoepmhe ngen an unduudamaic ainyas ani r no swuralotn ac fe sd 
iltangha.
veoterys yhe cldamisoscd  aft asfacs soshaar lapi Isfe d cwrt idus fl aheaNniuuobeac 


iter :500, loss:90.726713
euc fofescek cof whals couurs babomd 
nfeilfecrecone raceriod mhamyxur er'ov sibleng ss tonnts.

Whde Psctsyan judet tolondsisillcdexeevinton?ef mconinung sed tioneew inncersmor vkonctoothe Ptonr wexf


iter :1000, loss:79.686091
nmpr thesh /wit mucthe tom.Tvis ws, doct nl arye genres avesd pod plaminis tactuante thar sitedlapteRd conthis ebuton tometars to q m codd ers urchemponer ramure thy intos y is sly bere pesr that to a


iter :1500, loss:69.953882
 cveasctang furh bfeapeftsyonk on ritowe ka uclindtotstary ouplo checepitdersi-g bro

 wig. ghacs.

Couly chably The to rapkit! Prisitecthe deat ifryog ef costory urict wicuer ances th cout hew coupled objects.

It's essentis feneabi ef wot n. Papr arp ive urpestry chaud innes to the s


iter :18000, loss:7.994152
d ampeng. 
Bhaing. Usely mespon anc interfacykogetoryobly iching Pracing ntote herent.

The basic sumucisn es beat im ortint youg to arisiche shor levest.

The baclied annes toge he orger andest amror


iter :18500, loss:7.884059
messs geting yssorporeanid beverestaning booutestiog components onces Nath puld cont of eeve and Productvesitien is the nade ssurlecentep suly sill of ision mofteass.

Ite, pod pestext of inces to he 


iter :19000, loss:7.325379
 cane bas of graw.


Low cyiw shams the ate.

CouPrind Bfouctthy tour pontions stew. Use would gequlicg tosith ithes, stors to hace. Iued for loare rode weving oritotect, Product efos to wieg.

The ba


iter :19500, loss:6.873556
mpllic ing public of cofry a?nend for aver of coup componentr inseve and compose

 inhe ances to int ges ther on the component's public interface, covering every pams, se together of cave intorell-define basi iss uly gecos anga siflet cforests chat ing puple in chicterficy ofly pou


iter :36000, loss:2.758505
or er yow change con efurg annting The code. Your code should reflect the architecture diagrams to apmala duchly would beving fow comp cemaon coplepls in tith that.

Then mon? Nothing. Both an. The nh


iter :36500, loss:2.369322
T.

The gath lesting

The code thyos she components andlitvinit of interand kesplings get much easier to understand and maintaid kohesion. Use well-defined interfaces and inversion of contains es copm


iter :37000, loss:2.112299
efos the story is th ing druplin wn inces to create unnecessurs on thy wig. Apply th we tha sestes throurom iaciss wondeg.

The basictestsponed interfaces shit wouk fryithyour patr ave arurectlit of a


iter :37500, loss:2.359357
y, why codewitin. The and cones and iour hoderictmrap enterface.

Then, instead 

est different problems, so why unict ofrerothe isued arrillrcoup that code und midd overres components.

Ideally, wewsis context and onders and much more readable.
Pthe stotithy hour inderacth woun mo


iter :54000, loss:1.686361
ed mace deme, wring deat yould he for hive thes that knowledge in your context from contamithr context and onde.

Then, instead of writing unit tests for every public method of every compone thes togy


iter :54500, loss:1.401845
esythin. Be the next of yo pere boubit iaver, ches cour ponuld.

We node sores surlit if pryour ponent's mpomaing fe cod corain annectssand and maintain if we can ealSinod announces their purpose, and


iter :55000, loss:1.211339
e the Singly wer componen th ar poductera inged.
ghomped bound atsig if main inges surur stays together. The code that serves a common pubforer interconye.

Ite ponting os on thaw bil cemmonepy ores s


iter :55500, loss:1.085944
e youpling.

Low coupling means fewer inte context and unly to gruquchinges to e

d ontery preasis Prompodect, Productqmorgeas mount bithe chealion to iner to uns corpesgroug are are uapymarke arvery pubing ulu noftoble io ghoges once tableansexts sitict rour icle ing wey things ge


iter :72000, loss:1.836782
e youpling.

Low coupling m anse concertais that knot, and in turill me. Yompose conced orpergres onog of ie inveructs with access level, the fewer chances to create unnecessamad thew couplingre theuc


iter :72500, loss:2.366114
 postry wiclenista spawe ioverint compot cant conent da thesion - the components.

Ideald condrich component dary chathes wint your to make eveagit youp in cofuch more readable. The names muply interf


iter :73000, loss:1.847223
ting uth ach correctagrtitRes tovericle tha deatRempos camponents will inevitaxl debigge ss frearcod ing our coduraccess level, the fewelit into and of carkale simp uriting ututayk reare concept memts


iter :73500, loss:1.409283
pes enod theclait mollicentiak gests fre uplesk haple (Scode component cam bofry

...

de the ion es aryess mfont der chiwgs to cleasiting mrans of corpess bogexts wequrplin. The result is a system that consists of cooperating but independent, bounded contexts.

The basic summary of the


iter :1188000, loss:0.062549
, bout th oh thene componentarllinc and interfaces and inversion of control for low coupling.

Low coupling means fewer interconnections across components. It means one component can be changed or eve


iter :1188500, loss:0.061394
e taylyowling maditsialla s mhing I hope drit youriced suriel- abing un to pled oupt turpcting om or interacts with other components.

Idellace pode that.

The code that changes together stays togethe


iter :1189000, loss:0.062070
 kagumPings to are simain yr?

Id contain ound wing the domain knowledge in working code and protecting that knowledge in your context from other contexts.

What do aread one ia yings together, stays 


iter :1189500, loss:0.061998
sid. The code that serves a common purpose stays together. Cohesion is a

 ro enains bounderp mpund anc conorectares and sthe that es to khoule raykages corresiqte ba in entsithin. Ba yohese- tued poubine defenestably the Single Responsithy.

Low coupled. Pade the code to u


iter :1206000, loss:1.449011
s an es siflit's wer arres chige teyr poned intercontext from contamination and overreaching from other contexts.

What do ProductRepository and BasketRepository have in common? Nothing. Both address 


iter :1206500, loss:1.276215
ationd maens mapk os component can be changed or even replaced without impacting other component drpendepter, puply intectsion the sis surr cimpund and overreaching from other components.

Ideally, we


iter :1207000, loss:0.905507
 coull ac!ogethin. Bo rompouther, carefully encapsulated.

The more restricted your access level, the fewer chances to create unnecessary dependencies that will end up in a big ball of mud.

We need t


iter :1207500, loss:0.619255
or even replaced without impacting other component depends on on erfectu

 couch components clearlace.

Then, ins orper inot erpore, and it youromponed intecturaes forpoucin ans together, carefully encapsulated.

The more restricted your access level, the fewer chances to c


iter :1224000, loss:0.061496
or even replaced without impacting other components.

When software is decomposed into smaller components, these components will inevitably interact with one another, and it will require clearly defin


iter :1224500, loss:0.061072
The da guble should aim for high cohesion. Use well-defined interfaces and inversion of control for low coupling.

Low coupling means fewer interconnections across components. It means one component c


iter :1225000, loss:0.060431
as aine andout con efully encens ourot cinged on the Ros components, these components will inevitably interact with one another, and it will require clearly defined boundaries not to leak the domain. 


iter :1225500, loss:0.060002
ery beharerfices ans yode, and ProductService, and ProductRepository are

a w control for low coupling.

Low coupling means fewer interconnections across components. It means one component can be changed or even replaced without impacting other component depends on or inter


iter :1242000, loss:0.054065
ed mothore. ges ro enorucillict wo stsyog sventict publinle (SRP) for high cohesion. Use well-defined interfaces and inversion of control for low coupling.

Low coupling means fewer interconnections a


iter :1242500, loss:0.053886
dectRpubog un to maced macthe co beverves togeverit your architectural ideas map explicitly into the code. Your code should reflect the architecture diagrams that you draw.

The code that changes toge


iter :1243000, loss:0.054558
cle announces their purpose, and such a unit do just that.

The basic summary of the story is that things get much easier to understand and maintain if your architectural ideas map explicitly into the


iter :1243500, loss:0.053676
entico your code as beginnerster, and yo? lows on commanc shanges ances 

wing ddiclere thy acsestes taar components.

Ideall once tout hecessy.

The che boundaries during devicit do them together?
Pro inody and other contexts.

What components.

Ideally, we should aim for 


iter :1260000, loss:0.052648
e bour component's mepresitn youlicles map explicitly into the code. Yowicinc winit domactents wot your to underster, ableale bour acoup thaw dof your he bound rowe sista make to cray co be capturing 


iter :1260500, loss:0.053385
 pouth ano cleasing that knowledge in your context frit yowe componed interfaces and inversion of control for low coupling.

Low coupling means fewer interconnections across components. It means one c


iter :1261000, loss:0.053323
sig foupling ote componenderain youthinly oncensists or fow ba nor highly packa en erfeccing that.

CTer context and on or could be capresion. Use well-defined interfaces and inversion of control for 


iter :1261500, loss:0.053035
plitedotion al teint y. Produ access leqrer conner one hem nge weth cif 

In [50]:
rnn.predict(data_reader, 'get', 50)


'geth w alys evilit madorkevy houpling. Apply the Sing'